# Excombatants Classification using wavelets and MKL


In [1]:
#Librerías Necesarias Para el Código organizadas por tipo de llamado y orden alfabético*.
import csv
#import h5py
import scipy
import statistics
import time
import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd
import scipy.io as sio
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from scipy import signal
from scipy import spatial
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import LeaveOneOut
from sklearn import svm
from ckaweightedMKL1 import ckaweightedK
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score as acc
from sklearn.utils.class_weight import compute_class_weight
#from unipath import Path


## CARGA DE LAS CARACTRERÍSTICAS Y LAS ETIQUETAS BINARIZADAS.

In [2]:
np.set_printoptions(threshold=np.inf) #Imprime todos los valores de los arrays, sirve para visualizar mejor los datos


import pickle
y = np.ones(150)
y[50:100] = y[50:100]*2
y[100:150] = y[100:150]*3
label_matrix = y

num_valences= 3
num_rois = 8
num_channels= 62
num_trials= 50
#To assess the performance of the C in each emotion, later to evaluate is not necessary.





# Created functions to perform MKL clasiffication

In [3]:
def MixKer(K,eta):
    Kn= np.zeros_like((K[1]))
    for i in range(len(eta)):
        Kn= Kn + eta[i]*(K[i])
    return(Kn)

def KerRBF(X):
    d= spatial.distance.pdist(X)
    gamma= 1/(X.shape[1]*X.var())
    K= -spatial.distance.squareform(d)*gamma
    K= np.exp(K)
    return(K)
    
def KerLin(X):
    K = np.matmul(X,X.T)
    return(K)

def graficar(F1_test,Subjects,emocion,save):  
    Bandas=np.arange(1,5)
    Subject= np.arange(1,Subjects+1)
    contador=1
   

    plt.figure(contador,figsize=(6,5))
    plt.title('MKL Emocion: ')
    plt.bar(Subject, F1_test[:], align='center', alpha=0.5, color=['black', 'red', 'green', 'blue', 'cyan'])
    plt.ylim(0,1)
    plt.xlabel('Frequency Band')
    plt.ylabel('F1 Score')
    if save== 'y':
        plt.savefig('FiguresMKL_5/'+'C2_Improved')
    plt.show()
    contador+=1
    
    
def svc_param_selection(y, nfolds,Kernel_train_Wavelets):
    Cs = 1.**np.arange(0,1)
    #gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C':Cs}
    clf=svm.SVC(kernel='precomputed',class_weight = 'balanced')
    clf.fit(Kernel_train_Wavelets,y)
    grid_search = GridSearchCV(clf,param_grid,  cv=nfolds)
    grid_search.fit(Kernel_train, y)
    grid_search.best_params_
    return grid_search.best_params_



def Moda(N_Emociones,Subjects,Best_C_Matrix):
    Mode_C=np.zeros([N_Emociones])
    Mode_aux=np.zeros([Subjects,N_Emociones])
    Mode_aux2=np.zeros([Subjects,N_Emociones])
    for emo in range(0,N_Emociones):
        print('Emocion: ',emo)
        for sub in range(Subjects):
 
            a= Best_C_Matrix[sub,emo,:]
            a=a.tolist()
            Mode_aux[sub,emo]= max(a,key=a.count)           
      
        try:
            c= Mode_aux[:,emo]
            c=c.tolist()
            Mode_C[emo]=max(c,key=c.count)
        except:
            print('Nada3')
    
    print(' Mode Valence: ',Mode_C[0],'\n Mode Arousal: ',Mode_C[1],'\n Mode Dominance: ',Mode_C[2], '\n Mode Liking: ',Mode_C[3])
    return Mode_C



# SVM RBF Multiple Kernel Learning MKL

## Classification using 3 valences and 40 bands: 
### Here it was performed a classification for each valence and it was combined the bands and ROIs in a single dimension. 

In [4]:
import warnings
from sklearn.model_selection import KFold as KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
Valences=3
CrossVal= KFold(n_splits=30,shuffle=False)
N_Band= 40
N_Trials= 150
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
#N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs_2021/Features_Stats_Wavelets_'+wave_name[wave]+'_valences_bandrois'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,:,:]))
    
    F1_test_fold = np.zeros([N_folds])
    ACC_test_fold = np.zeros([N_folds])
    #SVM_Prediction_test=np.zeros([Valences,N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
    
    y_test = []
    print('Analizando Wavelet: '+wave_name[wave]) 

    for band in range(0,N_Band):
        scaler = MinMaxScaler()
        K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[band,:,:]))
        K[band,:,:] = scaler.fit_transform(K[band,:,:])
        K[band,:,:] = normalize(K[band,:,:])

    fold = 0

    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[1,:,:]),label_matrix):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
        ytrain2=np.expand_dims(ytrainMKL,axis=1)
        y_test.append(ytest)

        #Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[valence,band,train_index,:]), (Features_Stats_Wavelets[valence,band,test_index,:])

    #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

        Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
        Ktrain =  K[:,train_index,:]
        Ktrain = Ktrain[:,:,train_index]
        Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
        Ktest = K[:,test_index,:]
        Ktest = Ktest[:,:,train_index]


    #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

        #earlystop = EarlyStopping(
        #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
        #            patience=5,     #max number of acceptable negative steps
        #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
        #            metric='auc',   #the metric we monitor
        #            )

        #scheduler = ReduceOnWorsening()

        #mkl = GRAM(
        #max_iter=1000,          
        #learning_rate=.01,      
        #callbacks=[earlystop],
        #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
        #print(mkl)
        #mkl.predict(Ktest)


    #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

#             clf_Linear_Easy = EasyMKL(lam=0.5,multiclass_strategy='ova').fit(Ktrain, ytrain)	#a wrapper for averaging kernels
        #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
#             weights_train = clf_Linear_Easy.weights
        #print(weights_train)

        weights_train = ckaweightedK(Ktrain,ytrain)

        #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
        #weights_train = weights_train/np.sum(weights_train)

        #aux=np.ones(len(weights_train))
        #weights_train = aux-weights_train
        #print(weights_train)
        weights[fold]=weights_train
        Kernel_train= MixKer(Ktrain,weights_train)
        Kernel_test= MixKer(Ktest,weights_train)

    #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
        """"
        clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
        K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
        ##print(K_average)
        #print ('training EasyMKL...', end='')
        clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
        K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
        #print(K_Linear_average)
        Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
        Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
        """
    #---F1-score evaluation for training set------------------------------------------------------------------------------------

        #Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
        #Median= statistics.median(Distances)
        Parameters_grid = svc_param_selection(ytrain,5,Kernel_train)
        SVM_Classifier= svm.SVC(kernel='precomputed',C=Parameters_grid['C'],class_weight = 'balanced')
        SVM_Classifier.fit(Kernel_train,ytrain)
        SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
        SVM_Prediction_test= SVM_Classifier.predict(Kernel_test)
        F1_test_fold[fold] = f1_score(y_test[fold],SVM_Prediction_test,zero_division=1, average="micro")
#             ACC_test_fold[fold]= acc(y_test[fold],SVM_Prediction_test)
        print("y_test:",y_test[fold],"SVM:",SVM_Prediction_test)
        #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


        fold+=1

        #---F1-score evaluation for test set---------------------------------------------------------------------------------------

        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        F1_test = np.mean(F1_test_fold[:])
        ACC_test= np.mean(ACC_test_fold[:])
        STD_F1_test = np.std(F1_test_fold[:])
        STD_ACC_test = np.std(ACC_test_fold[:])
        print(F1_test)
    #graficar(F1_test,Subjects,emocion)
    
    wav= wave_name[wave]+'_valences_bandrois_easy_MKL'
    #Pickle Save F1 Score
    path_save= "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open(path_save+'Weights_'+wav+'.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open(path_save+'Weights_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

(40, 150, 8)
Analizando Wavelet: sym8
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.]
INICIO TEST
0.03333333333333333
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [2. 1. 2. 3. 2.]
INICIO TEST
0.04
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 2. 1. 1.]
INICIO TEST
0.06666666666666667
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.]
INICIO TEST
0.1
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.]
INICIO TEST
0.13333333333333333
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.]
INICIO TEST
0.16666666666666666
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.]
INICIO TEST
0.2
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.]
INICIO TEST
0.23333333333333334
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 2. 1. 1. 1.]
INICIO TEST
0.26
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 3. 3. 1.]
INICIO TEST
0.28
(145, 145) (145,)
y_test: [2. 2. 2. 2. 2.] SVM: [2. 1. 2. 3. 1.]
INIC

In [5]:
label_matrix

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])

## Classification using 3 valences and 5 bands: 
### Here it was performed a classification for each valence and it was combined the ROIs and statistical features in a single dimension, keeping the bands dimension. 

In [6]:
Features_Stats_Wavelets.shape

(40, 150, 8)

In [7]:
import warnings
from sklearn.model_selection import KFold as KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
CrossVal= KFold(n_splits=30,shuffle=False)
N_Band= 5
N_Trials= 150
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
#N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs_2021/Features_Stats_Wavelets_'+wave_name[wave]+'_valences_bands'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,:,:]))
    
    F1_test_fold = np.zeros([N_folds])
    ACC_test_fold = np.zeros([N_folds])
    #SVM_Prediction_test=np.zeros([Valences,N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
    
    y_test = []
    print('Analizando Wavelet: '+wave_name[wave]) 

    for band in range(0,N_Band):
        scaler = MinMaxScaler()
        K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[band,:,:]))
        K[band,:,:] = scaler.fit_transform(K[band,:,:])
        K[band,:,:] = normalize(K[band,:,:])

    fold = 0

    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[1,:,:])):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
        ytrain2=np.expand_dims(ytrainMKL,axis=1)
        y_test.append(ytest)

        #Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[valence,band,train_index,:]), (Features_Stats_Wavelets[valence,band,test_index,:])

    #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

        Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
        Ktrain =  K[:,train_index,:]
        Ktrain = Ktrain[:,:,train_index]
        Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
        Ktest = K[:,test_index,:]
        Ktest = Ktest[:,:,train_index]


    #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

        #earlystop = EarlyStopping(
        #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
        #            patience=5,     #max number of acceptable negative steps
        #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
        #            metric='auc',   #the metric we monitor
        #            )

        #scheduler = ReduceOnWorsening()

        #mkl = GRAM(
        #max_iter=1000,          
        #learning_rate=.01,      
        #callbacks=[earlystop],
        #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
        #print(mkl)
        #mkl.predict(Ktest)


    #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

#         clf_Linear_Easy = EasyMKL(lam=1).fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
#         weights_train = clf_Linear_Easy.weights
        #print(weights_train)

        weights_train = ckaweightedK(Ktrain,ytrain)

        #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
        #weights_train = weights_train/np.sum(weights_train)

        #aux=np.ones(len(weights_train))
        #weights_train = aux-weights_train
        #print(weights_train)
        weights[fold]=weights_train
        Kernel_train= MixKer(Ktrain,weights_train)
        Kernel_test= MixKer(Ktest,weights_train)

    #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
        """"
        clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
        K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
        ##print(K_average)
        #print ('training EasyMKL...', end='')
        clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
        K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
        #print(K_Linear_average)
        Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
        Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
        """
    #---F1-score evaluation for training set------------------------------------------------------------------------------------

        #Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
        #Median= statistics.median(Distances)
        Parameters_grid = svc_param_selection(ytrain,5,Kernel_train)
        SVM_Classifier= svm.SVC(kernel='precomputed',C=Parameters_grid['C'],class_weight = 'balanced')
        SVM_Classifier.fit(Kernel_train,ytrain)
        SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
        SVM_Prediction_test= SVM_Classifier.predict(Kernel_test)
        F1_test_fold[fold] = f1_score(y_test[fold],SVM_Prediction_test,zero_division=1, average="micro")
        ACC_test_fold[fold]= acc(y_test[fold],SVM_Prediction_test)
        print("y_test:",y_test[fold],"SVM:",SVM_Prediction_test,"ACC:",ACC_test_fold[fold],"F1:",F1_test_fold[fold],"C:",Parameters_grid['C'])
        #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


        fold+=1

        #---F1-score evaluation for test set---------------------------------------------------------------------------------------

        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        F1_test = np.mean(F1_test_fold[:])
        ACC_test= np.mean(ACC_test_fold[:])
        STD_F1_test = np.std(F1_test_fold[:])
        STD_ACC_test = np.std(ACC_test_fold[:])
        print(ACC_test)
    #graficar(F1_test,Subjects,emocion)
    
    wav= wave_name[wave]+'_valences_bands_easy_MKL'
    #Pickle Save F1 Score
    path_save= "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open(path_save+'Weights_'+wav+'.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open(path_save+'Weights_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

(5, 150, 64)
Analizando Wavelet: sym8
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.03333333333333333
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.06666666666666667
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.1
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.13333333333333333
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.16666666666666666
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.2
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.23333333333333334
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] ACC: 1.0 F1: 1.0 C: 1.0
INICIO TEST
0.26666666666666666
(145, 145) (145,)
y_test: 

In [8]:
weights

array([[ 2.23738254e-01, -1.40438497e-01,  5.77300101e-01,
         1.37126302e-01, -7.60354650e-01],
       [ 3.29379917e-01, -1.33213689e-01,  4.93614762e-01,
         1.14672783e-01, -7.85466488e-01],
       [ 3.27060636e-01, -1.10361482e-01,  4.88668455e-01,
         1.00627962e-01, -7.94939518e-01],
       [ 4.94717698e-02,  1.47528498e-01,  5.56571347e-01,
         8.80083959e-02, -8.11338859e-01],
       [ 3.08400300e-01, -8.13975630e-02,  4.88339787e-01,
         1.08892753e-01, -8.04941186e-01],
       [ 1.82398395e-01, -4.71972440e-02,  5.61574844e-01,
         1.31016717e-01, -7.94966389e-01],
       [ 1.74666724e-01,  4.83980205e-02,  5.21828585e-01,
         1.04610498e-01, -8.26982913e-01],
       [ 1.56213002e-01, -7.92205280e-02,  5.54299760e-01,
         1.84395198e-01, -7.92509806e-01],
       [ 2.53144855e-01, -1.64209083e-01,  5.60220973e-01,
         1.36622859e-01, -7.59236271e-01],
       [ 1.60669781e-01, -1.15425629e-01,  4.94875127e-01,
         2.88222491e-01

## Classification using 3 valences and 8 Rois: 
### Here it was performed a classification for each valence and it was combined bands and statistical features in a single dimension, keeping the rois dimension. 

In [9]:
import warnings
from sklearn.model_selection import KFold as KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
Valences=3
CrossVal= KFold(n_splits=30,shuffle=False)
N_Band= 8
N_Trials= 150
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
#N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs_2021/Features_Stats_Wavelets_'+wave_name[wave]+'_valences_rois'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,:,:]))
    
    F1_test_fold = np.zeros([N_folds])
    ACC_test_fold = np.zeros([N_folds])
    #SVM_Prediction_test=np.zeros([Valences,N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
    
    y_test = []
    print('Analizando Wavelet: '+wave_name[wave]) 

    for band in range(0,N_Band):
        scaler = MinMaxScaler()
        K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[band,:,:]))
        K[band,:,:] = scaler.fit_transform(K[band,:,:])
        K[band,:,:] = normalize(K[band,:,:])

    fold = 0

    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[1,:,:])):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
        ytrain2=np.expand_dims(ytrainMKL,axis=1)
        y_test.append(ytest)

        #Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[valence,band,train_index,:]), (Features_Stats_Wavelets[valence,band,test_index,:])

    #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

        Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
        Ktrain =  K[:,train_index,:]
        Ktrain = Ktrain[:,:,train_index]
        Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
        Ktest = K[:,test_index,:]
        Ktest = Ktest[:,:,train_index]


    #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

        #earlystop = EarlyStopping(
        #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
        #            patience=5,     #max number of acceptable negative steps
        #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
        #            metric='auc',   #the metric we monitor
        #            )

        #scheduler = ReduceOnWorsening()

        #mkl = GRAM(
        #max_iter=1000,          
        #learning_rate=.01,      
        #callbacks=[earlystop],
        #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
        #print(mkl)
        #mkl.predict(Ktest)


    #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

#         clf_Linear_Easy = EasyMKL(lam=1).fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
#         weights_train = clf_Linear_Easy.weights
        #print(weights_train)

        weights_train = ckaweightedK(Ktrain,ytrain)

        #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
        #weights_train = weights_train/np.sum(weights_train)

        #aux=np.ones(len(weights_train))
        #weights_train = aux-weights_train
        #print(weights_train)
        weights[fold]=weights_train
        Kernel_train= MixKer(Ktrain,weights_train)
        Kernel_test= MixKer(Ktest,weights_train)

    #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
        """"
        clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
        K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
        ##print(K_average)
        #print ('training EasyMKL...', end='')
        clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
        K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
        #print(K_Linear_average)
        Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
        Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
        """
    #---F1-score evaluation for training set------------------------------------------------------------------------------------

        #Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
        #Median= statistics.median(Distances)
        Parameters_grid = svc_param_selection(ytrain,5,Kernel_train)
        SVM_Classifier= svm.SVC(kernel='precomputed',C=Parameters_grid['C'],class_weight = 'balanced')
        SVM_Classifier.fit(Kernel_train,ytrain)
        SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
        SVM_Prediction_test= SVM_Classifier.predict(Kernel_test)
        F1_test_fold[fold] = f1_score(y_test[fold],SVM_Prediction_test,zero_division=1, average="micro")
        ACC_test_fold[fold]= acc(y_test[fold],SVM_Prediction_test)
        print("y_test:",y_test[fold],"SVM:",SVM_Prediction_test,"F1:",F1_test_fold[fold],"C:",Parameters_grid['C'])
        #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


        fold+=1

    #---F1-score evaluation for test set---------------------------------------------------------------------------------------

    print('INICIO TEST')
    #Calcular F1_test
    #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
    F1_test = np.mean(F1_test_fold[:])
    ACC_test= np.mean(ACC_test_fold[:])
    STD_F1_test = np.std(F1_test_fold[:])
    STD_ACC_test = np.std(ACC_test_fold[:])
    print(ACC_test)
    
    wav= wave_name[wave]+'_valences_rois_easy_MKL'
    #Pickle Save F1 Score
    path_save= "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open(path_save+'Weights_'+wav+'.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open(path_save+'Weights_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

(8, 150, 40)
Analizando Wavelet: sym8
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] F1: 1.0 C: 1.0
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] F1: 1.0 C: 1.0
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] F1: 1.0 C: 1.0
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] F1: 1.0 C: 1.0
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] F1: 1.0 C: 1.0
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] F1: 1.0 C: 1.0
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] F1: 1.0 C: 1.0
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] F1: 1.0 C: 1.0
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] F1: 1.0 C: 1.0
(145, 145) (145,)
y_test: [1. 1. 1. 1. 1.] SVM: [1. 1. 1. 1. 1.] F1: 1.0 C: 1.0
(145, 145) (145,)
y_test: [2. 2. 2. 2. 2.] SVM: [2. 2. 2. 2. 2.] F1: 1.0 C: 1.0
(145, 145) (145,)
y_test: [2. 2. 2. 2. 2.] SVM: [2. 2. 2. 2. 2.] F1: 1.0 C: 1.0
(1

In [10]:
weights.shape

(30, 8)

## Classification using 5 bands and no-valences: 
### Here it was performed a classification mixing the Valences, ROIs and statistical features in a single dimension, keeping the bands dimension. 

In [11]:
import warnings
from sklearn.model_selection import KFold as KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
Valences=3
CrossVal= KFold(n_splits=25,shuffle=False)
N_Band= 5
N_Trials= 50
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
#N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs_2021/Features_Stats_Wavelets_'+wave_name[wave]+'_valencesrois'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,:,:]))
    
    F1_test_fold = np.zeros([N_folds])
    ACC_test_fold = np.zeros([N_folds])
    #SVM_Prediction_test=np.zeros([Valences,N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
    

    y_test = []
    print('Analizando Wavelet: '+wave_name[wave]) 

    for band in range(0,N_Band):
        scaler = MinMaxScaler()
        K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[band,:,:]))
        K[band,:,:] = scaler.fit_transform(K[band,:,:])
        K[band,:,:] = normalize(K[band,:,:])

    fold = 0

    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[1,:,:]),label_matrix):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
        ytrain2=np.expand_dims(ytrainMKL,axis=1)
        y_test.append(ytest)

        #Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[valence,band,train_index,:]), (Features_Stats_Wavelets[valence,band,test_index,:])

    #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

        Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
        Ktrain =  K[:,train_index,:]
        Ktrain = Ktrain[:,:,train_index]
        Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
        Ktest = K[:,test_index,:]
        Ktest = Ktest[:,:,train_index]


    #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

        #earlystop = EarlyStopping(
        #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
        #            patience=5,     #max number of acceptable negative steps
        #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
        #            metric='auc',   #the metric we monitor
        #            )

        #scheduler = ReduceOnWorsening()

        #mkl = GRAM(
        #max_iter=1000,          
        #learning_rate=.01,      
        #callbacks=[earlystop],
        #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
        #print(mkl)
        #mkl.predict(Ktest)


    #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

        clf_Linear_Easy = EasyMKL(lam=1).fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
        weights_train = clf_Linear_Easy.weights
        #print(weights_train)

        #weights_train = ckaweightedK(Ktrain,ytrain)

        #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
        #weights_train = weights_train/np.sum(weights_train)

        #aux=np.ones(len(weights_train))
        #weights_train = aux-weights_train
        #print(weights_train)
        weights[fold]=weights_train
        Kernel_train= MixKer(Ktrain,weights_train)
        Kernel_test= MixKer(Ktest,weights_train)

    #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
        """"
        clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
        K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
        ##print(K_average)
        #print ('training EasyMKL...', end='')
        clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
        K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
        #print(K_Linear_average)
        Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
        Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
        """
    #---F1-score evaluation for training set------------------------------------------------------------------------------------

        #Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
        #Median= statistics.median(Distances)
        Parameters_grid = svc_param_selection(ytrain,5,Kernel_train)
        SVM_Classifier= svm.SVC(kernel='precomputed',C=Parameters_grid['C'],class_weight = 'balanced')
        SVM_Classifier.fit(Kernel_train,ytrain)
        SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
        SVM_Prediction_test= SVM_Classifier.predict(Kernel_test)
        F1_test_fold[fold] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
        ACC_test_fold[fold]= acc(y_test[fold].T[0],SVM_Prediction_test)
        print("y_test:",y_test[fold].T[0],"SVM:",SVM_Prediction_test,"ACC:",ACC_test_fold[fold],"F1:",F1_test_fold[fold],"C:",Parameters_grid['C'])
        #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


        fold+=1

        #---F1-score evaluation for test set---------------------------------------------------------------------------------------

        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        F1_test = np.mean(F1_test_fold[:])
        ACC_test= np.mean(ACC_test_fold[:])
        STD_F1_test = np.std(F1_test_fold[:])
        STD_ACC_test = np.std(ACC_test_fold[:])
        print(ACC_test)
    #graficar(F1_test,Subjects,emocion)
    
    wav= wave_name[wave]+'_valencesrois_easy_MKL'
    #Pickle Save F1 Score
    path_save= "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open(path_save+'Weights_'+wav+'.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open(path_save+'Weights_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

(4, 50, 192)
Analizando Wavelet: sym8


IndexError: index 4 is out of bounds for axis 0 with size 4

## Classification using 8 Rois and no-valences: 
### Here it was performed a classification mixing the Valences, Bands and statistical features in a single dimension, keeping the Rois dimension. 

In [ ]:
import warnings
from sklearn.model_selection import KFold as KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
Valences=3
CrossVal= KFold(n_splits=25,shuffle=False)
N_Band= 8
N_Trials= 50
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
#N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs_2021/Features_Stats_Wavelets_'+wave_name[wave]+'_valencesbands'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,:,:]))
    
    F1_test_fold = np.zeros([N_folds])
    ACC_test_fold = np.zeros([N_folds])
    #SVM_Prediction_test=np.zeros([Valences,N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
    

    y_test = []
    print('Analizando Wavelet: '+wave_name[wave]) 

    for band in range(0,N_Band):
        scaler = MinMaxScaler()
        K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[band,:,:]))
        K[band,:,:] = scaler.fit_transform(K[band,:,:])
        K[band,:,:] = normalize(K[band,:,:])

    fold = 0

    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[1,:,:]),label_matrix):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
        ytrain2=np.expand_dims(ytrainMKL,axis=1)
        y_test.append(ytest)

        #Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[valence,band,train_index,:]), (Features_Stats_Wavelets[valence,band,test_index,:])

    #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

        Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
        Ktrain =  K[:,train_index,:]
        Ktrain = Ktrain[:,:,train_index]
        Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
        Ktest = K[:,test_index,:]
        Ktest = Ktest[:,:,train_index]


    #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

        #earlystop = EarlyStopping(
        #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
        #            patience=5,     #max number of acceptable negative steps
        #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
        #            metric='auc',   #the metric we monitor
        #            )

        #scheduler = ReduceOnWorsening()

        #mkl = GRAM(
        #max_iter=1000,          
        #learning_rate=.01,      
        #callbacks=[earlystop],
        #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
        #print(mkl)
        #mkl.predict(Ktest)


    #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

        clf_Linear_Easy = EasyMKL(lam=1).fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
        weights_train = clf_Linear_Easy.weights
        #print(weights_train)

        #weights_train = ckaweightedK(Ktrain,ytrain)

        #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
        #weights_train = weights_train/np.sum(weights_train)

        #aux=np.ones(len(weights_train))
        #weights_train = aux-weights_train
        #print(weights_train)
        weights[fold]=weights_train
        Kernel_train= MixKer(Ktrain,weights_train)
        Kernel_test= MixKer(Ktest,weights_train)

    #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
        """"
        clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
        K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
        ##print(K_average)
        #print ('training EasyMKL...', end='')
        clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
        K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
        #print(K_Linear_average)
        Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
        Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
        """
    #---F1-score evaluation for training set------------------------------------------------------------------------------------

        #Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
        #Median= statistics.median(Distances)
        Parameters_grid = svc_param_selection(ytrain,5,Kernel_train)
        SVM_Classifier= svm.SVC(kernel='precomputed',C=Parameters_grid['C'],class_weight = 'balanced')
        SVM_Classifier.fit(Kernel_train,ytrain)
        SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
        SVM_Prediction_test= SVM_Classifier.predict(Kernel_test)
        F1_test_fold[fold] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
        ACC_test_fold[fold]= acc(y_test[fold].T[0],SVM_Prediction_test)
        print("y_test:",y_test[fold].T[0],"SVM:",SVM_Prediction_test,"ACC:",ACC_test_fold[fold],"F1:",F1_test_fold[fold],"C:",Parameters_grid['C'])
        #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


        fold+=1

        #---F1-score evaluation for test set---------------------------------------------------------------------------------------

        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        F1_test = np.mean(F1_test_fold[:])
        ACC_test= np.mean(ACC_test_fold[:])
        STD_F1_test = np.std(F1_test_fold[:])
        STD_ACC_test = np.std(ACC_test_fold[:])
        print(ACC_test)
    #graficar(F1_test,Subjects,emocion)
    
    wav= wave_name[wave]+'_valencesbands_easy_MKL'
    #Pickle Save F1 Score
    path_save= "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open(path_save+'Weights_'+wav+'.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open(path_save+'Weights_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

## Classification using 40 bands and no-valences: 
### Here it was performed a classification mixing the Valences, ROIs and statistical features in a single dimension, keeping the bands dimension. 

In [ ]:
import warnings
from sklearn.model_selection import KFold as KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
Valences=3
CrossVal= KFold(n_splits=25,shuffle=False)
N_Band= 40
N_Trials= 50
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
#N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs_2021/Features_Stats_Wavelets_'+wave_name[wave]+'_bandrois'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,:,:]))
    
    F1_test_fold = np.zeros([N_folds])
    ACC_test_fold = np.zeros([N_folds])
    #SVM_Prediction_test=np.zeros([Valences,N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
    

    y_test = []
    print('Analizando Wavelet: '+wave_name[wave]) 

    for band in range(0,N_Band):
        scaler = MinMaxScaler()
        K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[band,:,:]))
        K[band,:,:] = scaler.fit_transform(K[band,:,:])
        K[band,:,:] = normalize(K[band,:,:])

    fold = 0

    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[1,:,:]),label_matrix):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
        ytrain2=np.expand_dims(ytrainMKL,axis=1)
        y_test.append(ytest)

        #Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[valence,band,train_index,:]), (Features_Stats_Wavelets[valence,band,test_index,:])

    #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

        Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
        Ktrain =  K[:,train_index,:]
        Ktrain = Ktrain[:,:,train_index]
        Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
        Ktest = K[:,test_index,:]
        Ktest = Ktest[:,:,train_index]


    #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

        #earlystop = EarlyStopping(
        #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
        #            patience=5,     #max number of acceptable negative steps
        #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
        #            metric='auc',   #the metric we monitor
        #            )

        #scheduler = ReduceOnWorsening()

        #mkl = GRAM(
        #max_iter=1000,          
        #learning_rate=.01,      
        #callbacks=[earlystop],
        #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
        #print(mkl)
        #mkl.predict(Ktest)


    #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

        #clf_Linear_Easy = EasyMKL(lam=1).fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
        #weights_train = clf_Linear_Easy.weights
        #print(weights_train)

        weights_train = ckaweightedK(Ktrain,ytrain)

        #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
        #weights_train = weights_train/np.sum(weights_train)

        #aux=np.ones(len(weights_train))
        #weights_train = aux-weights_train
        #print(weights_train)
        weights[fold]=weights_train
        Kernel_train= MixKer(Ktrain,weights_train)
        Kernel_test= MixKer(Ktest,weights_train)

    #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
        """"
        clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
        K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
        ##print(K_average)
        #print ('training EasyMKL...', end='')
        clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
        K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
        #print(K_Linear_average)
        Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
        Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
        """
    #---F1-score evaluation for training set------------------------------------------------------------------------------------

        #Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
        #Median= statistics.median(Distances)
        Parameters_grid = svc_param_selection(ytrain,5,Kernel_train)
        SVM_Classifier= svm.SVC(kernel='precomputed',C=Parameters_grid['C'],class_weight = 'balanced')
        SVM_Classifier.fit(Kernel_train,ytrain)
        SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
        SVM_Prediction_test= SVM_Classifier.predict(Kernel_test)
        F1_test_fold[fold] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
        ACC_test_fold[fold]= acc(y_test[fold].T[0],SVM_Prediction_test)
        print("y_test:",y_test[fold].T[0],"SVM:",SVM_Prediction_test,"ACC:",ACC_test_fold[fold],"F1:",F1_test_fold[fold],"C:",Parameters_grid['C'])
        #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


        fold+=1

        #---F1-score evaluation for test set---------------------------------------------------------------------------------------

        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        F1_test = np.mean(F1_test_fold[:])
        ACC_test= np.mean(ACC_test_fold[:])
        STD_F1_test = np.std(F1_test_fold[:])
        STD_ACC_test = np.std(ACC_test_fold[:])
        print(ACC_test)
    #graficar(F1_test,Subjects,emocion)
    
    wav= wave_name[wave]+'_bandrois_easy_MKL'
    #Pickle Save F1 Score
    path_save= "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open(path_save+'Weights_'+wav+'.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open(path_save+'Weights_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

In [ ]:
Features_Stats_Wavelets.shape

In [ ]:
print(ACC_test)
print(STD_F1_test)
print(weights)

# UNUSED

# 1)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
CrossVal= LeaveOneOut()
N_Band= 5
N_Trials= 50
N_Rois = 8
C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs/Features_Stats_Wavelets_'+wave_name[wave]+'_valencesrois'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,:,:]))
    
    
    SVM_Prediction_test=np.zeros([N_folds])
    y_test = np.zeros([N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
  
    print('Analizando Wavelet: '+wave_name[wave]+' Valencia : ') 

    for band in range(0,N_Band):
        K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[band,:,:]))

        fold = 0

    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[1,:,:])):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        y_test[fold] = ytest
        ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
        ytrain2=np.expand_dims(ytrainMKL,axis=1)
        y_test[fold] = ytest

        Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[band,train_index,:]), (Features_Stats_Wavelets[band,test_index,:])

    #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

        Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
        Ktrain =  K[:,train_index,:]
        Ktrain = Ktrain[:,:,train_index]
        Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
        Ktest = K[:,test_index,:]
        Ktest = Ktest[:,:,train_index]


    #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

        #earlystop = EarlyStopping(
        #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
        #            patience=5,     #max number of acceptable negative steps
        #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
        #            metric='auc',   #the metric we monitor
        #            )

        #scheduler = ReduceOnWorsening()

        #mkl = GRAM(
        #max_iter=1000,          
        #learning_rate=.01,      
        #callbacks=[earlystop],
        #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
        #print(mkl)
        #mkl.predict(Ktest)


    #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

        clf_Linear_Easy = EasyMKL(lam=1).fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
        weights_train = clf_Linear_Easy.weights
        #print(weights_train)

        #weights_train = ckaweightedK(Ktrain,ytrain)

        #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
        #weights_train = weights_train/np.sum(weights_train)

        #aux=np.ones(len(weights_train))
        #weights_train = aux-weights_train
        #print(weights_train)
        weights[fold]=weights_train
        Kernel_train= MixKer(Ktrain,weights_train)
        Kernel_test= MixKer(Ktest,weights_train)

    #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
        """"
        clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
        K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
        ##print(K_average)
        #print ('training EasyMKL...', end='')
        clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
        K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
        #print(K_Linear_average)
        Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
        Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
        """
    #---F1-score evaluation for training set------------------------------------------------------------------------------------

        Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
        Median= statistics.median(Distances)
        #Gamma= 1/(2*(Median**2))
        Gamma= 1/(Xtrain.shape[1]*Xtrain.var())
        Parameters_grid = svc_param_selection(ytrain,5,Gamma,Kernel_train)
        SVM_Classifier= svm.SVC(kernel='precomputed',gamma=Gamma,C=Parameters_grid['C'],class_weight = 'balanced')
        SVM_Classifier.fit(Kernel_train,ytrain)
        SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
        SVM_Prediction_test[fold]= SVM_Classifier.predict(Kernel_test)
        #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


        fold+=1

        #---F1-score evaluation for test set---------------------------------------------------------------------------------------

    print('INICIO TEST')
    #Calcular F1_test
    #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
    F1_test = f1_score(y_test[:],SVM_Prediction_test[:])
    ACC_test= acc(y_test[:],SVM_Prediction_test[:])
    print(F1_test)
    #graficar(F1_test,Subjects,emocion)
    
    wav= wave_name[wave]+'_valencesrois_easy'
    #Pickle Save F1 Score
    import pickle
    file= open('Wavelets_Results_ROIs/F1_Score_'+wav+'_MKL.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open('Wavelets_Results_ROIs/F1_Score_'+wav+'_MKL.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open('Wavelets_Results_ROIs/ACC_Score_'+wav+'_MKL.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open('Wavelets_Results_ROIs/ACC_Score_'+wav+'_MKL.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open('Wavelets_Results_ROIs/Weights_'+wav+'_MKL.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open('Wavelets_Results_ROIs/Weights_'+wav+'_MKL.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

# 2)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
CrossVal= LeaveOneOut()
N_Band= 40
N_Trials= 50
C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,Subjects,Best_C_Matrix)
N_cs = len(C)


#---Iterations for each Emotion and subject--------------------------------------------------------------------------

wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5):
    File= open('Wavelets_Results_ROIs/Features_Stats_Wavelets_'+wave_name[wave]+'_bandrois'+'.pckl', 'rb')
    Features_Stats_Wavelets = pickle.load(File)
    File.close()
    N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,:,:]))
    
    SVM_Prediction_test=np.zeros([N_folds])
    y_test = np.zeros([N_folds])
    K = np.zeros([N_Band,N_Trials,N_Trials])
    weights=np.zeros([N_folds,N_Band])

    print(Features_Stats_Wavelets.shape)
    
   
    print('Analizando Wavelet: '+wave_name[wave]) 

    for band in range(0,N_Band):
        K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[band,:,:]))

        fold = 0

    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[1,:,:])):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        y_test[fold] = ytest
        ytrainMKL, ytestMKL = label_matrix[train_index], label_matrix[test_index]
        ytrain2=np.expand_dims(ytrainMKL,axis=1)
        y_test[fold] = ytest

        Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[band,train_index,:]), (Features_Stats_Wavelets[band,test_index,:])

    #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

        Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
        Ktrain =  K[:,train_index,:]
        Ktrain = Ktrain[:,:,train_index]
        Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
        Ktest = K[:,test_index,:]
        Ktest = Ktest[:,:,train_index]


    #--- Multiple Kernel Learning GRAM---------------------------------------------------------------------

        #earlystop = EarlyStopping(
        #            Ktrain, ytrain,      #validation data, KL is a validation kernels list
        #            patience=5,     #max number of acceptable negative steps
        #            cooldown=1,     #how ofter we run a measurement, 1= every optimization step
        #            metric='auc',   #the metric we monitor
        #            )

        #scheduler = ReduceOnWorsening()

        #mkl = GRAM(
        #max_iter=1000,          
        #learning_rate=.01,      
        #callbacks=[earlystop],
        #scheduler=ReduceOnWorsening()).fit(Ktrain, ytrain)
        #print(mkl)
        #mkl.predict(Ktest)


    #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

        clf_Linear_Easy = EasyMKL(lam=1).fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #clf_Linear_Easy = AverageMKL().fit(Ktrain,ytrain)
        weights_train = clf_Linear_Easy.weights
        #print(weights_train)

        #weights_train = ckaweightedK(Ktrain,ytrain)

        #weights_train = np.random.uniform(low=0.0, high=1.0, size=N_Band)
        #weights_train = weights_train/np.sum(weights_train)

        #aux=np.ones(len(weights_train))
        #weights_train = aux-weights_train
        #print(weights_train)
        weights[fold]=weights_train
        Kernel_train= MixKer(Ktrain,weights_train)
        Kernel_test= MixKer(Ktest,weights_train)

    #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
        """"
        clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
        K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
        ##print(K_average)
        #print ('training EasyMKL...', end='')
        clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
        #print ('done')
        #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
        K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
        #print(K_Linear_average)
        Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
        Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
        """
    #---F1-score evaluation for training set------------------------------------------------------------------------------------

        Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
        Median= statistics.median(Distances)
        Parameters_grid = svc_param_selection(ytrain,5,Gamma,Kernel_train)
        SVM_Classifier= svm.SVC(kernel='precomputed',gamma=Gamma,C=Parameters_grid['C'],class_weight = 'balanced')
        SVM_Classifier.fit(Kernel_train,ytrain)
        SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
        SVM_Prediction_test[fold]= SVM_Classifier.predict(Kernel_test)
        #F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')


        fold+=1

    #---F1-score evaluation for test set---------------------------------------------------------------------------------------

    print('INICIO TEST')
    #Calcular F1_test
    #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
    F1_test = f1_score(y_test[:],SVM_Prediction_test[:])
    ACC_test = acc(y_test[:],SVM_Prediction_test[:])
    print(F1_test)
#graficar(F1_test,Subjects,emocion)
    
    wav= wave_name[wave]+'_bandrois_easy'
    #Pickle Save F1 Score
    import pickle
    file= open('Wavelets_Results_ROIs/F1_Score_'+wav+'_MKL.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open('Wavelets_Results_ROIs/F1_Score_'+wav+'_MKL.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open('Wavelets_Results_ROIs/ACC_Score_'+wav+'_MKL.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open('Wavelets_Results_ROIs/ACC_Score_'+wav+'_MKL.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle Save Weights
    file= open('Wavelets_Results_ROIs/Weights_'+wav+'_MKL.pckl', 'wb')
    pickle.dump(weights,file)
    file.close()

    File= open('Wavelets_Results_ROIs/Weights_'+wav+'_MKL.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == weights).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
    print('SE GUARDÓ EL ARCHIVO '+wav) 

In [ ]:
weights_train

In [ ]:
File= open('Wavelets_Results/F1_Score_'+'bior3.3_sparse'+'_MKL.pckl', 'rb')
F1_test = pickle.load(File)
File.close()
print(F1_test)

In [ ]:
#F1_test.tofile('Test_5\F1_Test_KRBF_C3_MKL_Improved'+'.txt')
#Best_C_Matrix.tofile('Test_5\Best_C_Matrix_C2_MKL'+'.txt')

# GRÁFICA DEL F1_SCORE PARA CADA SUJETO EN CADA BANDA Y PARA CADA C 


In [ ]:
import warnings
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------
Subjects= 32
CrossVal= LeaveOneOut()
N_folds= CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[1,1,:,:]))
N_Band= 6
N_Trials= 40
N_Emociones=4
C = 2.**np.arange(-15,19)
N_cs = len(C)
Best_C_Matrix=np.zeros([Subjects,N_Emociones,N_folds])
F1_train = np.zeros([Subjects,N_Emociones,N_folds,N_cs])
F1_test = np.zeros([Subjects,N_Emociones])
SVM_Prediction_test=np.zeros([Subjects,N_Emociones,N_folds])
y_test = np.zeros([Subjects,N_folds])
K = np.zeros([N_Band,N_Trials,N_Trials])

#---Iterations for each Emotion and subject--------------------------------------------------------------------------

for emo in range(N_Emociones):
    for sub in range(Subjects):
        print('Análisis Sujeto: ',sub, 'Emoción: ',emo)
        if (sub==26 and emo==2):
            sub+=1
            
        for band in range(0,N_Band):
            K[band,:,:] = KerRBF(np.squeeze(Features_Stats_Wavelets[sub,band,:,:]))

        fold = 0
        emocion=emo
        for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[sub,1,:,:])):
            ytrain, ytest = label_threshold_matrix_5[sub,train_index,emocion], label_threshold_matrix_5[sub,test_index,emocion]
            y_test[sub,fold] = ytest
            ytrainMKL, ytestMKL = label_matrix[sub,train_index,emocion], label_matrix[sub,test_index,emocion]
            ytrain2=np.expand_dims(ytrainMKL,axis=1)
            y_test[sub,fold] = ytest

            Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[sub,band,train_index,:]), (Features_Stats_Wavelets[sub,band,test_index,:])

        #---Definition of kernels variables for the multiple kernel learning--------------------------------------------------------

            Ktrain = np.zeros([N_Band,len(train_index),len(train_index)])
            Ktrain =  K[:,train_index,:]
            Ktrain = Ktrain[:,:,train_index]
            Ktest = np.zeros([N_Band,len(test_index),len(train_index)])
            Ktest = K[:,test_index,:]
            Ktest = Ktest[:,:,train_index]

        #---Multiple Kernel Learning Custom Functions---------------------------------------------------------------------------

            #print(Ktrain.shape, ytrain2.shape)
            weights_train = ckaweightedK(Ktrain,ytrain2)
            #print(weights_train)
            Kernel_train= MixKer(Ktrain,weights_train)
            Kernel_test= MixKer(Ktest,weights_train)

        #---Multiple Kernel Learning------------------------------------------------------------------------------------------------
            """"
            clf_Linear_Average = AverageMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
            #print ('done')
            #print(clf_Linear_Average.weights)			#print the weights of the combination of base kernels
            K_Linear_average = clf_Linear_Average.ker_matrix	#the combined kernel matrix
            ##print(K_average)
            #print ('training EasyMKL...', end='')
            clf_Linear_Easy = EasyMKL().fit(Ktrain,ytrain)	#a wrapper for averaging kernels
            #print ('done')
            #print(clf_Linear_Easy.weights)			#print the weights of the combination of base kernels
            K_Linear_easy = clf_Linear_Easy.ker_matrix	#the combined kernel matrix
            #print(K_Linear_average)
            Ktrain = MixKer(Ktrain,clf_Linear_Easy.weights)
            Ktest = MixKer(Ktest,clf_Linear_Easy.weights)
            """
        #---F1-score evaluation for training set------------------------------------------------------------------------------------

            Best_C= -10000
            Best_F1=0
            for c in range(len(C)):
                Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
                Median= statistics.median(Distances)
                Gamma= 1/(Xtrain.shape[1]*Xtrain.var())
                SVM_Classifier= svm.SVC(kernel='precomputed',C=C[c])
                SVM_Classifier.fit(Kernel_train,ytrain)
                SVM_Prediction_train= SVM_Classifier.predict(Kernel_train)
                SVM_Prediction_test[sub,emo,fold]= SVM_Classifier.predict(Kernel_test)
                F1_train[sub,emo,fold,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')

                if F1_train[sub,emo,fold,c]>Best_F1:
                    Best_C_Matrix[valence,fold]=C[c]
                    Best_C= C[c]
                    Best_F1=F1_train[valence,fold,c]
            fold+=1



        #---F1-score evaluation for test set---------------------------------------------------------------------------------------

        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        F1_test[sub,emo] = f1_score(y_test[sub,:],SVM_Prediction_test[sub,emo,:],average='macro')
        #print(F1_test[sub,band,c])
        #time.sleep(4)
    graficar(F1_test,Subjects,emocion)
                

In [ ]:
%matplotlib notebook
graficar(F1_test,Subjects,3)
Moda(N_Emociones,Subjects,Best_C_Matrix)

In [ ]:
#F1_test.tofile('Test\F1_Test_Customlib_RBF_Lineal_C3_'+Emocion[emocion]+'.txt')

In [ ]:
band=['Theta','Low Alpha', 'High Alpha', 'Alha', 'Betha']


for i in range(0,7):
    plt.figure(i)
    #plt.title('Kernel for '+ band[i])
    plt.imshow(Ktrain[i],cmap= plt.cm.get_cmap('OrRd'))
    #plt.savefig('DEAP_Figures/Kernel_'+str(i))

In [ ]:
plt.imshow(Kernel_train,cmap= plt.cm.get_cmap('OrRd'))
#plt.savefig('DEAP_Figures/Final_Kernel')
print(weights_train) 


In [ ]:
ensayo=np.matmul(ytrain2,ytrain2.T)
plt.imshow(ensayo,cmap= plt.cm.get_cmap('OrRd'))

In [ ]:
%matplotlib notebook
x=np.arange(0,496)
y=np.arange(0,40)
X,Y= np.meshgrid(x,y)
z=Y*X
print(Feature_MSC_Matrix_C2[0,0].shape)
fig= plt.figure()
ax= fig.gca(projection='3d')
ax.contour(X,Y,Feature_MSC_Matrix_C2[0,0,:,:])